In [1]:
from pyspark.sql import *

In [3]:
spark=SparkSession.builder.getOrCreate()

In [4]:
df=spark.read.csv("titanic.csv",header=True)

In [5]:
df.show(3)

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male| 22|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|  7.925|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
only showing top 3 rows



In [8]:
df.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       0|  545|
|       1|  342|
+--------+-----+



In [ ]:
import pyspark.sql.functions as f

In [35]:
df.groupBy(f.col("Survived")).count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       0|  545|
|       1|  342|
+--------+-----+



In [30]:
schema="Survived int,Pclass int,Name String,Sex String,\
Age int,`Siblings/Spouses Aboard` int,`Parents/Children Aboard`  int ,Fare double"

In [31]:
df_schema=spark.read.csv('titanic.csv',header=True,schema=schema)

In [32]:
df_schema.dtypes

[('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'int'),
 ('Siblings/Spouses Aboard', 'int'),
 ('Parents/Children Aboard', 'int'),
 ('Fare', 'double')]

In [33]:
df_schema.show(3)

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male| 22|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|  7.925|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
only showing top 3 rows



In [34]:
df_schema.groupBy("Survived").max("Age").show()

+--------+--------+
|Survived|max(Age)|
+--------+--------+
|    null|    null|
|       1|      80|
|       0|      74|
+--------+--------+



In [39]:
df_schema.groupBy(["Survived","Sex"]).min("Age").show()

+--------+------+--------+
|Survived|   Sex|min(Age)|
+--------+------+--------+
|       0|female|       2|
|    null|  null|    null|
|       1|  male|       1|
|       1|female|       1|
|       0|  male|       1|
+--------+------+--------+



In [79]:
agg_dict={"Age":"mean","Fare":"max","`Siblings/Spouses Aboard`":"std"}

In [80]:
df.groupBy("Survived").agg(agg_dict).show()

+--------+-------------------------------+------------------+---------+
|Survived|stddev(Siblings/Spouses Aboard)|          avg(Age)|max(Fare)|
+--------+-------------------------------+------------------+---------+
|       0|              1.292248779344448| 30.13853211009174|       90|
|       1|             0.7086875190740347|28.408391812865496|     93.5|
+--------+-------------------------------+------------------+---------+



In [109]:
df.groupBy(["Survived","Sex"]).agg({"*":f.countDistinct()}).show()

+--------+------+----------------------+--------+
|Survived|   Sex|count(DISTINCT Pclass)|count(1)|
+--------+------+----------------------+--------+
|       1|  male|                     3|     109|
|       1|female|                     3|     233|
|       0|female|                     3|      81|
|       0|  male|                     3|     464|
+--------+------+----------------------+--------+



In [124]:
df.groupBy(["Survived","Sex"]).agg(*[f.max("Pclass").alias("dist_pclass"),f.min("Age").alias("min_age")]).show()

+--------+------+-----------+-------+
|Survived|   Sex|dist_pclass|min_age|
+--------+------+-----------+-------+
|       1|  male|          3|   0.42|
|       1|female|          3|   0.75|
|       0|female|          3|     10|
|       0|  male|          3|      1|
+--------+------+-----------+-------+



In [ ]:
import pyspark.sql

In [88]:
df_date=df_schema.selectExpr("*","current_date as date_field")

In [89]:
df_date.dtypes

[('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'int'),
 ('Siblings/Spouses Aboard', 'int'),
 ('Parents/Children Aboard', 'int'),
 ('Fare', 'double'),
 ('date_field', 'date')]

In [98]:
df_date.groupby(f.year("date_field")).count().show()

+----------------+-----+
|year(date_field)|count|
+----------------+-----+
|            2019|  887|
+----------------+-----+



In [94]:
dftest=df.selectExpr("'2019-11-11' AS string_date" ).groupBy(f.month("string_date")).count().show()

+------------------+-----+
|month(string_date)|count|
+------------------+-----+
|                11|  887|
+------------------+-----+



In [127]:
df_schema.groupBy("Survived").agg(f.collect_list("Pclass")).show()

+--------+--------------------+
|Survived|collect_list(Pclass)|
+--------+--------------------+
|       1|[1, 3, 1, 3, 2, 3...|
|       0|[3, 3, 3, 1, 3, 3...|
+--------+--------------------+



In [128]:
df_schema.groupBy("Survived").agg(f.collect_set("Pclass")).show()

+--------+-------------------+
|Survived|collect_set(Pclass)|
+--------+-------------------+
|       1|          [1, 2, 3]|
|       0|          [1, 2, 3]|
+--------+-------------------+



In [133]:
df_date.filter("Survived=1").show()

+--------+------+--------------------+------+---+-----------------------+-----------------------+--------+----------+
|Survived|Pclass|                Name|   Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|    Fare|date_field|
+--------+------+--------------------+------+---+-----------------------+-----------------------+--------+----------+
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0| 71.2833|2019-12-01|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|   7.925|2019-12-01|
|       1|     1|Mrs. Jacques Heat...|female| 35|                      1|                      0|    53.1|2019-12-01|
|       1|     3|Mrs. Oscar W (Eli...|female| 27|                      0|                      2| 11.1333|2019-12-01|
|       1|     2|Mrs. Nicholas (Ad...|female| 14|                      1|                      0| 30.0708|2019-12-01|
|       1|     3|Miss. Marguerite ...|female|  4|       

In [135]:
df_date.filter(f.col("Survived")==1).show(3)

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+----------+
|Survived|Pclass|                Name|   Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|date_field|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+----------+
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|2019-12-01|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|  7.925|2019-12-01|
|       1|     1|Mrs. Jacques Heat...|female| 35|                      1|                      0|   53.1|2019-12-01|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+----------+
only showing top 3 rows



In [138]:
df_date.filter((f.col("Survived")==1) & (f.col("Sex")=="male")).show(3)

+--------+------+--------------------+----+---+-----------------------+-----------------------+----+----------+
|Survived|Pclass|                Name| Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|Fare|date_field|
+--------+------+--------------------+----+---+-----------------------+-----------------------+----+----------+
|       1|     2|Mr. Charles Eugen...|male| 23|                      0|                      0|13.0|2019-12-01|
|       1|     2|Mr. Lawrence Beesley|male| 34|                      0|                      0|13.0|2019-12-01|
|       1|     1|Mr. William Thomp...|male| 28|                      0|                      0|35.5|2019-12-01|
+--------+------+--------------------+----+---+-----------------------+-----------------------+----+----------+
only showing top 3 rows



In [145]:
df_date.filter("Pclass = 1 and Age=23").show()

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+----------+
|Survived|Pclass|                Name|   Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|date_field|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+----------+
|       1|     1|Miss. Mabel Helen...|female| 23|                      3|                      2|  263.0|2019-12-01|
|       1|     1|Mr. William Bertr...|  male| 23|                      0|                      1|63.3583|2019-12-01|
|       1|     1|Miss. Marjorie Ne...|female| 23|                      1|                      0|113.275|2019-12-01|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+----------+



In [147]:
df_date.filter("Sex = 'male' and Name like '%John%'").show()

+--------+------+--------------------+----+---+-----------------------+-----------------------+--------+----------+
|Survived|Pclass|                Name| Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|    Fare|date_field|
+--------+------+--------------------+----+---+-----------------------+-----------------------+--------+----------+
|       0|     3|Mr. William John ...|male| 30|                      0|                      0|    8.05|2019-12-01|
|       0|     3|Mr. David John Ba...|male| 22|                      0|                      0|    8.05|2019-12-01|
|       0|     2|Mr. William John ...|male| 29|                      1|                      0|    21.0|2019-12-01|
|       0|     3|Mr. John Hatfield...|male| 44|                      0|                      1|    16.1|2019-12-01|
|       0|     3|Mr. John Viktor B...|male| 26|                      0|                      0|   7.775|2019-12-01|
|       1|     3|Master. Frank Joh...|male|  9|                      0| 

In [153]:
df_date[(df_date.Pclass==1) & (df_date.Sex=="male")].show()

+--------+------+--------------------+----+---+-----------------------+-----------------------+--------+----------+
|Survived|Pclass|                Name| Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|    Fare|date_field|
+--------+------+--------------------+----+---+-----------------------+-----------------------+--------+----------+
|       0|     1|Mr. Timothy J McC...|male| 54|                      0|                      0| 51.8625|2019-12-01|
|       1|     1|Mr. William Thomp...|male| 28|                      0|                      0|    35.5|2019-12-01|
|       0|     1|Mr. Charles Alexa...|male| 19|                      3|                      2|   263.0|2019-12-01|
|       0|     1|Don. Manuel E Uru...|male| 40|                      0|                      0| 27.7208|2019-12-01|
|       0|     1|Mr. Edgar Joseph ...|male| 28|                      1|                      0| 82.1708|2019-12-01|
|       0|     1|Mr. Alexander Osk...|male| 42|                      1| 